In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 3

@author: yaning
"""

import numpy as np
import importlib
import matplotlib.pyplot as plt
import torch
from torch.distributions import Normal, Uniform
from tqdm import tqdm


# my own class files
import with_learning.learning_NN.Receptors as Receptors
import with_learning.learning_NN.Network as Network


importlib.reload(Receptors)
importlib.reload(Network)

<module 'with_learning.learning_NN.Network' from '/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Network.py'>

In [6]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [7]:
pointCount = 5000
deltaTms = 0.05
times = np.arange(pointCount) * deltaTms
initial_Vm = 1.3458754117369027

# Neuron: deltaTms, I, Vm, Name
neuron_input_0 = Network.Neuron(deltaTms, 0, initial_Vm, "input_0")
neuron_input_1 = Network.Neuron(deltaTms, 0, initial_Vm, "input_1")
neuron_input_2 = Network.Neuron(deltaTms, 0, initial_Vm, "input_2")

neuron_excite_main = Network.Neuron(deltaTms, 0, initial_Vm, "excite_main")
neuron_excite_sub = Network.Neuron(deltaTms, 0, initial_Vm, "excite_sub")

neuron_inhibit_main = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_main")
neuron_inhibit_sub = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_sub")

neuron_output = Network.Neuron(deltaTms, 0, initial_Vm, "output")

neurons = [neuron_input_0, neuron_input_1, neuron_input_2, neuron_excite_main, neuron_excite_sub, 
        neuron_inhibit_main, neuron_inhibit_sub, neuron_output]

neuron_names = ["input_0", "input_1", "input_2",
                "excite_main", "excite_sub", "inhibit_main", "inhibit_sub", "output"]





# create synapse/connection (send neuron, receive neuron)
control = Network.Control(deltaTms, initial_Vm)




#*********************full layer***************************
# ----------------first input layer------------------------
control.create_synapse(neuron_input_0, neuron_excite_main, "AMPA")
control.create_synapse(neuron_input_1, neuron_excite_main, "AMPA")
control.create_synapse(neuron_input_2, neuron_excite_main, "AMPA")

control.create_synapse(neuron_input_0, neuron_inhibit_main, "GABA")
control.create_synapse(neuron_input_1, neuron_inhibit_main, "GABA")
control.create_synapse(neuron_input_2, neuron_inhibit_main, "GABA")



# ----------------self recurrent layer----------------
control.create_synapse(neuron_excite_main, neuron_excite_sub, "AMPA+NMDA")
control.create_synapse(neuron_excite_sub, neuron_excite_main, "AMPA+NMDA")

control.create_synapse(neuron_inhibit_main, neuron_inhibit_sub, "GABA")
control.create_synapse(neuron_inhibit_sub, neuron_inhibit_main, "GABA")

# --------------between excitatory and inhibitory----------------
control.create_synapse(neuron_excite_main, neuron_inhibit_main, "AMPA+NMDA")
control.create_synapse(neuron_inhibit_main, neuron_excite_main, "GABA")


# ----------------output layer----------------------
control.create_synapse(neuron_excite_main, neuron_output, "AMPA")


# recording arrays




In [8]:
# get the input neuron firing patterns
path = "/home/yaning/Documents/Spiking_NN/with_learning/"
input_pattern = np.load(path + "dataset.npy") 
output_pattern = np.load(path + "output.npy")
input_pattern = input_pattern[1]

In [9]:

infer_params = Receptors.LigandGatedChannelFactory.infer_params
infer_names = Receptors.LigandGatedChannelFactory.infer_names
factor = 30

In [10]:
num_samples = 100
results = []

for i in range(num_samples):
    all_voltages = []
    pure_sample = []
    for j in range(len(infer_params)):
        # using j only for separate with and without factor
        temp_pure_sample = Normal(0, 2).sample()
        pure_sample.append(temp_pure_sample)
        if j <= 9:
            infer_params[infer_names[j]] = sigmoid(temp_pure_sample)
        else:
            infer_params[infer_names[j]] = factor*sigmoid(temp_pure_sample)
    
    for t in range(pointCount):
        if input_pattern[0,t]:
            neuron_input_0.sending_signal()
            neuron_input_0.fire_tstep.append(t)
            
        if input_pattern[1,t]:
            neuron_input_1.sending_signal()
            neuron_input_1.fire_tstep.append(t)
            
        if input_pattern[2,t]:
            neuron_input_2.sending_signal()
            neuron_input_2.fire_tstep.append(t)


        # update the synapse states then each neuron
        for neuron in neurons[3:]:
            neuron.check_firing(t)
            neuron.update()
            all_voltages.append(neuron.Vm)
            
        # set the synapse states back to 0
        for synapse in control.all_synapses:
            synapse.state = 0
    if max(all_voltages) <= 200:
        results.append(pure_sample)
        break



/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:79: RuntimeWarning: overflow encountered in exp
  self.m.alpha = .1*((25-self.Vm) / (np.exp((25-self.Vm)/10)-1))
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:80: RuntimeWarning: overflow encountered in exp
  self.m.beta = 4*np.exp(-self.Vm/18)
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:81: RuntimeWarning: overflow encountered in exp
  self.n.alpha = .01 * ((10-self.Vm) / (np.exp((10-self.Vm)/10)-1))
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:82: RuntimeWarning: overflow encountered in exp
  self.n.beta = .125*np.exp(-self.Vm/80)
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:83: RuntimeWarning: overflow encountered in exp
  self.h.alpha = .07*np.exp(-self.Vm/20)
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:84: RuntimeWarning: overflow encountered in exp
  self.h.beta = 1/(np